In [1]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version
import time
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd /content/drive/Shareddrives/rajc990330@gmail.com/HDSP/DeepFusion/deep-fusion-admm
import tensorflow as tf

Fri Oct  8 21:52:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!ls

callbacks.py		__pycache__	   sensing_layers.py
custom_loss_metrics.py	read_datasets.py   unrolling_layers.py
deep_prior_networks.py	regularizers.py    unrolling_network.py
generate_datasets	results
main.py			run_df_admm.ipynb


In [3]:
import tensorflow as tf
import os

from unrolling_network import *
from custom_loss_metrics import *
from callbacks import *
from read_datasets import *

#---------- Params -----------------------
M = 256
L = 31
ds = 0.5
dl = 0.5
batch_size = 1
stages_t = 10
snr = 30
noise = True
reg_param = 0.5
sig_param = 10
opt_H = True

# Network configuration
config_loss = 'ML_T'
config_t = 'T_reg_s'  # ['NT', 'T_reg_n', 'T_reg_s']

print('Training model: Loss: ' + config_loss + ' | CA training: ' + config_t)
if config_t == 'T_reg_n':
    type_reg = 'Normal'
    in_reg_param = 0.5
else:
    type_reg = 'sig'
    in_reg_param = 30


# Select dataset
dataset = "cave"
dataset_dict = 'im'

train_path = "/content/drive/Shareddrives/rajc990330@gmail.com/HDSP/Datasets/"+ dataset +"/train"
val_path = "/content/drive/Shareddrives/rajc990330@gmail.com/HDSP/Datasets/"+ dataset +"/validation"
test_path = "/content/drive/Shareddrives/rajc990330@gmail.com/HDSP/Datasets/"+ dataset +"/test"



Training model: Loss: ML_T | CA training: T_reg_s


In [4]:
[train_gen, val_gen, test_gen] = generate_dataset(batch_size, M, L, train_path, val_path, test_path, dataset_dict)

# Callbacks
path = 'results/results_admm/'
csv_file = path + 'results_admm.csv'

model_path = path + 'model_admm.h5'

try:
    os.mkdir(path)
except OSError as error:
    print(error)

save_model_callback_all = save_each_epoch(model_path)

callbacks = [save_model_callback_all,
             tf.keras.callbacks.CSVLogger(csv_file, separator=',', append=False)]

lr = 0.001

optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                     name='Adam')


Training images:  21
Validation images:  5
Test images:  5
[Errno 17] File exists: 'results/results_admm/'


In [ ]:

model = unrollingNetwork(input_dim=(M, M, L), HO=(0.25, 0.5, 0.25), ds=ds, stage_t=stages_t, shots=1, batch_size=batch_size, dl=dl,noise=noise,snr=snr,opt_H=opt_H,reg_param=reg_param,sig_param=sig_param,type_reg=type_reg)

rho_l = 1
rho_s = 1
model.compile(optimizer=optimizer, loss=loss_unrolling(rho_l=rho_l,rho_s=rho_s), metrics=psnr_metric)

# model.summary()

h = model.fit(x=train_gen, validation_data=val_gen,verbose=1, epochs=300, callbacks=[callbacks])

Trainable MCFA
Epoch 1/300
21/21 [==============================] - 176s 4s/step - loss: 3899.6768 - add_loss: 1.9110 - X0_loss: 1.9686 - X1_loss: 1.9839 - X2_loss: 2.0787 - X3_loss: 2.0459 - X4_loss: 2.0593 - X5_loss: 2.0394 - X6_loss: 2.0420 - X7_loss: 2.0605 - X8_loss: 2.0218 - X9_loss: 2.0207 - add_psnr_metric: 12.9405 - X0_psnr_metric: 12.9145 - X1_psnr_metric: 12.9075 - X2_psnr_metric: 12.8715 - X3_psnr_metric: 12.8765 - X4_psnr_metric: 12.8588 - X5_psnr_metric: 12.9098 - X6_psnr_metric: 12.9018 - X7_psnr_metric: 12.8712 - X8_psnr_metric: 12.9746 - X9_psnr_metric: 12.9754 - val_loss: 3547.3657 - val_add_loss: 1.0873 - val_X0_loss: 1.0847 - val_X1_loss: 1.0845 - val_X2_loss: 1.0807 - val_X3_loss: 1.0810 - val_X4_loss: 1.0807 - val_X5_loss: 1.0720 - val_X6_loss: 1.0709 - val_X7_loss: 1.0714 - val_X8_loss: 1.0575 - val_X9_loss: 1.0562 - val_add_psnr_metric: 15.9575 - val_X0_psnr_metric: 15.9817 - val_X1_psnr_metric: 15.9848 - val_X2_psnr_metric: 16.0168 - val_X3_psnr_metric: 16.0150